# Simple Data Check

Basic queries to understand what's in the database.

In [ ]:
import sys
import os
import pandas as pd
import pyodbc
import warnings

# Add project root to Python path
project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)

from config.database_config import get_connection_string

warnings.filterwarnings('ignore')

# Connect to database
conn = pyodbc.connect(get_connection_string())
print("Connected to database")

## Step 1: Check if we have any price data at all

In [ ]:
query = "SELECT COUNT(*) as TotalRecords FROM T_CommodityPrice"
result = pd.read_sql(query, conn)
print(f"Total records in T_CommodityPrice: {result['TotalRecords'].iloc[0]}")

## Step 2: Check metals

In [ ]:
query = "SELECT * FROM M_Metal"
metals = pd.read_sql(query, conn)
print("All metals in database:")
print(metals)

## Step 3: Check tenor types

In [ ]:
query = "SELECT * FROM M_TenorType"
tenors = pd.read_sql(query, conn)
print("All tenor types in database:")
print(tenors)

## Step 4: Check recent price data (any data)

In [ ]:
query = """
SELECT TOP 20 
    TradeDate, 
    MetalID, 
    TenorTypeID, 
    SettlementPrice
FROM T_CommodityPrice 
WHERE SettlementPrice IS NOT NULL
ORDER BY TradeDate DESC
"""

recent = pd.read_sql(query, conn)
print("Recent price data (any metal/tenor):")
print(recent)

## Step 5: Join metals and prices to see what we have

In [ ]:
query = """
SELECT TOP 20
    p.TradeDate,
    m.MetalCode,
    m.ExchangeCode,
    p.TenorTypeID,
    p.SettlementPrice
FROM T_CommodityPrice p
INNER JOIN M_Metal m ON p.MetalID = m.MetalID
WHERE p.SettlementPrice IS NOT NULL
ORDER BY p.TradeDate DESC
"""

joined = pd.read_sql(query, conn)
print("Price data with metal info:")
print(joined)

## Step 6: Full join to see tenor names

In [ ]:
query = """
SELECT TOP 20
    p.TradeDate,
    m.MetalCode,
    m.ExchangeCode,
    t.TenorTypeName,
    p.SettlementPrice
FROM T_CommodityPrice p
INNER JOIN M_Metal m ON p.MetalID = m.MetalID
INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
WHERE p.SettlementPrice IS NOT NULL
ORDER BY p.TradeDate DESC
"""

full_data = pd.read_sql(query, conn)
print("Full data with all info:")
print(full_data)

## Step 7: Check what tenor types actually contain 'Generic'

In [ ]:
query = """
SELECT DISTINCT t.TenorTypeName
FROM T_CommodityPrice p
INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
WHERE t.TenorTypeName LIKE '%Generic%'
"""

generic_tenors = pd.read_sql(query, conn)
print("Tenor types containing 'Generic':")
print(generic_tenors)

## Step 8: Check what tenor types contain 'Future'

In [ ]:
query = """
SELECT DISTINCT t.TenorTypeName
FROM T_CommodityPrice p
INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
WHERE t.TenorTypeName LIKE '%Future%'
"""

future_tenors = pd.read_sql(query, conn)
print("Tenor types containing 'Future':")
print(future_tenors)

## Step 9: Check all unique tenor types that have data

In [ ]:
query = """
SELECT DISTINCT t.TenorTypeName
FROM T_CommodityPrice p
INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
WHERE p.SettlementPrice IS NOT NULL
ORDER BY t.TenorTypeName
"""

all_tenors_with_data = pd.read_sql(query, conn)
print("All tenor types that have price data:")
print(all_tenors_with_data)

## Step 10: Get some actual data for visualization

In [ ]:
# Based on what we found, let's get some actual data
query = """
SELECT 
    p.TradeDate,
    m.MetalCode,
    m.ExchangeCode,
    t.TenorTypeName,
    p.SettlementPrice,
    p.Volume,
    p.OpenInterest
FROM T_CommodityPrice p
INNER JOIN M_Metal m ON p.MetalID = m.MetalID
INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
WHERE p.SettlementPrice IS NOT NULL
    AND p.TradeDate >= DATEADD(day, -30, GETDATE())
ORDER BY p.TradeDate DESC, m.ExchangeCode, t.TenorTypeName
"""

actual_data = pd.read_sql(query, conn)
print(f"\nFound {len(actual_data)} records in last 30 days")
if len(actual_data) > 0:
    print("\nFirst 10 records:")
    print(actual_data.head(10))
    
    print("\nUnique exchanges:")
    print(actual_data['ExchangeCode'].unique())
    
    print("\nUnique tenor types:")
    print(actual_data['TenorTypeName'].unique())
    
    print("\nData count by exchange:")
    print(actual_data['ExchangeCode'].value_counts())
else:
    print("No data found in last 30 days")

In [ ]:
conn.close()
print("Database connection closed")